# Scenario Report (Public Version)

In [ ]:
# Copyright (c) 2021 *****************. All rights reserved.
# Licensed under the Apache License, Version 2.0, which is in the LICENSE file.

from pathlib import Path
import pandas as pd
import plotly.express as px
import plotly
import plotly.graph_objects as go 
import numpy as np

#get the name of the current directory to specify the scenario name and identify the output directory
scenario_name = str(Path.cwd()).split('\\')[-1]
if scenario_name == 'inputs':
    data_dir = Path.cwd() / '../outputs/'
    inputs_dir = Path.cwd() / '../inputs/'
    scenario_name = 'N/A'
else:
    data_dir = Path.cwd() / f'../../outputs/{scenario_name}/'
    inputs_dir = Path.cwd() / f'../../inputs/{scenario_name}/'

#define formatting options/functions for outputs
pd.options.display.float_format = '{:,.2f}'.format
def format_currency(x): 
    try:
        formatted = '$ {:,.2f}'.format(x)
    except ValueError:
        formatted = x
    return formatted
def format_percent(x): 
    try:
        formatted = '{:,.2f}%'.format(x)
    except ValueError:
        formatted = x
    return formatted

#allow the notebook to display plots in html report
###################################################
plotly.offline.init_notebook_mode()

print(f'Scenario Name: {scenario_name}')

## Portfolio Renewable Percentage

In [ ]:
# Calculate Time-coincident renewable % and annual renewable %
##############################################################

# load load balance data    
load_balance = pd.read_csv(data_dir / 'load_balance.csv', parse_dates=True)

# get the list of generation column names that actually exist in the dataframe
generation_columns_list = ['ZoneTotalGeneratorDispatch','ZoneTotalVariableGeneration','ZoneTotalStorageDispatch']
generation_columns_available_list = [i for i in load_balance.columns if i in generation_columns_list]

# Calculate total generation from all sources
load_balance['Total_Generation'] = load_balance[generation_columns_available_list].sum(axis=1) 
load_balance['Total_Generation_No_Storage'] = load_balance[['ZoneTotalGeneratorDispatch','ZoneTotalVariableGeneration']].sum(axis=1) 

# get the list of load column names that actually exist in the dataframe
load_columns_list = ['zone_demand_mw','ZoneTotalStorageCharge','ShiftDemand']
load_columns_available_list = [i for i in load_balance.columns if i in load_columns_list]

# Calculate total demand from all sources
load_balance['Total_Demand'] = load_balance[load_columns_available_list].sum(axis=1) 

# calculate the time-coincident generation
load_balance['Time_Coincident_Generation'] = load_balance[['Total_Generation','Total_Demand']].min(axis=1)
load_balance['Time_Coincident_Generation_No_Storage'] = load_balance[['Total_Generation_No_Storage','zone_demand_mw']].min(axis=1)

# TODO: Determine if SystemPower is ever used when not needed, which would affect the calculation
#calculate the time-coincident renewable percentage
try:
    tc_percent_renewable = (1 - load_balance['SystemPower'].sum() / load_balance['zone_demand_mw'].sum()) * 100
except KeyError:
    # if the SystemPower data is missing, then it is likely a 100% Time coincident scenario
    tc_percent_renewable = load_balance['Time_Coincident_Generation'].sum() / load_balance['Total_Demand'].sum() * 100

# calculate the time-coincident renewable % ignoring storage
tc_no_storage_percent_renewable = load_balance['Time_Coincident_Generation_No_Storage'].sum() / load_balance['zone_demand_mw'].sum() * 100

#tc_no_storage_percent_renewable = load_balance['Time_Coincident_Generation'].sum() / load_balance['zone_demand_mw'].sum() * 100
annual_percent_renewable = load_balance['Total_Generation_No_Storage'].sum() / load_balance['zone_demand_mw'].sum() * 100

print(f'Time-coincident renewable percentage (with storage dispatch): {np.round(tc_percent_renewable, decimals=2)}%')
print(f'Time-coincident renewable percentage (without storage dispatch): {np.round(tc_no_storage_percent_renewable, decimals=2)}%')
print(f'Annual volumetric renewable percentage: {annual_percent_renewable.round(decimals=2)}%')

## Generator Portfolio
The sunburst chart describes the built portfolio at various levels of detail, which shows how the outer rings relate to the inner rings
- Inner circle: contract status (contracted or additional project)
- Middle ring: technology type (e.g. solar, wind, ...)
- Outer ring: specific project name

For example, individual projects in the outer ring belong to a specific technology type in the middle ring, which can either be part of the existing/contracted portfolio, or the additional portfolio.


In [ ]:
# Prepare data for sunburst chart of portfolio
##############################################

#load capacity costs
portfolio = pd.read_csv(data_dir / 'gen_cap.csv', usecols=['generation_project','GenCapacity'])
portfolio = portfolio[portfolio['GenCapacity'] > 0]
portfolio = portfolio.rename(columns={'GenCapacity':'MW'})

# load data about predetermined generators
existing = pd.read_csv(inputs_dir / 'gen_build_predetermined.csv', usecols=['GENERATION_PROJECT']).rename(columns={'GENERATION_PROJECT':'generation_project'})


# add column indicating which generators are contracted or additional builds

existing['Status'] = 'Contracted'
portfolio = portfolio.merge(existing, how='left', on='generation_project').fillna('Additional')


#split the prefix of the generator name from the rest of the string
tech_column = [i.split('_')[0] for i in portfolio['generation_project']]
generator_name = [' '.join(i.split('_')[1:]) for i in portfolio['generation_project']]

#create a new tecnhology column based on the prefix and drop the generator name
portfolio['Technology'] = tech_column
portfolio['generation_project'] = generator_name

portfolio['MW'] = portfolio['MW'].round(1)

portfolio = portfolio.sort_values(by=['Status','Technology'])


portfolio_sunburst = px.sunburst(portfolio, path=['Status','Technology'], values='MW', color='Technology', color_discrete_map={'HYDRO':'Purple',
 'ONWIND':'Blue',
 'OFFWIND':'Navy',
 'PV':'Yellow',
 'PVHYBRID':'Yellow',
 'CSP':'Orange',
 'GEO':'Sienna',
 'STORAGE':'Green',
 'STORAGEHYBRID':'Green',
 '(?)':'Black'},
 width=1000, height=1000,
 title='Energy Portfolio by Technology Type, and Contract Status (MW)')
portfolio_sunburst.update_traces(textinfo='label+value')
portfolio_sunburst.show()

## Delivered Energy Mix
This is similar to a power content label, showing the percentage of energy from each source used to serve load.

In [ ]:
total_demand = load_balance['zone_demand_mw'].sum()
try:
    system_power_consumed = load_balance['SystemPower'].sum()
except KeyError:
    system_power_consumed = 0
generated_power_consumed = total_demand - system_power_consumed

# Calculate generated energy mix
generation_mix = pd.read_csv(data_dir / 'dispatch_annual_summary.csv', usecols=['gen_energy_source','Energy_GWh_typical_yr'])
# calculate the % generation from each technology
generation_mix['Energy_GWh_typical_yr'] = generation_mix['Energy_GWh_typical_yr'] / generation_mix['Energy_GWh_typical_yr'].sum()
# calculate the total MWh of consumed power from each source
generation_mix['Energy_GWh_typical_yr'] = generation_mix['Energy_GWh_typical_yr'] * generated_power_consumed
# rename the columns
generation_mix = generation_mix.rename(columns={'gen_energy_source':'Source','Energy_GWh_typical_yr':'Delivered MWh'})
# add a row for system power
generation_mix = generation_mix.append({'Source':'System_Power','Delivered MWh':system_power_consumed}, ignore_index=True)


generation_mix['Delivered MWh'] = generation_mix['Delivered MWh'].round(decimals=0)

# only keep non-zero data
generation_mix = generation_mix[generation_mix['Delivered MWh'] > 0]

# rename water to Hydro
generation_mix = generation_mix.replace('Water','Hydro')

delivered_energy_pie = px.pie(generation_mix, values='Delivered MWh', names='Source', title='Delivered Energy Mix (MWh)', color='Source', color_discrete_map={'Hydro':'Purple',
 'Onshore_Wind':'Blue',
 'Offshore_Wind':'Navy',
 'Solar':'Yellow',
 'Geothermal':'Sienna',
 'System_Power':'Red'},
width=600, height=600
 )
delivered_energy_pie.update_traces(textinfo='percent+label+value')
delivered_energy_pie.show()


## Resource Adequacy Open Position

In [ ]:
try:
    ra_open = pd.read_csv(data_dir / 'RA_open_position.csv')
    
    #where RA Position is negative, it indicates an open position
    ra_open['Color'] = np.where(ra_open["RA_Position_MW"]<0, 'Open Position', 'Excess RA')

    #create a plot of monthly RA position
    monthly_ra_open_fig = px.bar(ra_open, x='Month', y='RA_Position_MW', facet_col='RA_Requirement', color='Color', color_discrete_map={'Excess RA':'green', 'Open Position':'red'}, title='Monthly RA position by requirement')
    monthly_ra_open_fig.for_each_annotation(lambda a: a.update(text=a.text.replace("RA_Requirement=", "")))
    
except FileNotFoundError:
    pass

In [ ]:
try:
    monthly_ra_open_fig.show()
except:
    print('Resource Adequacy was not considered in this scenario')

# Supply and Demand Balance

In [ ]:
# Set up data for dispatch timeseries graph
###########################################

dispatch = pd.read_csv(data_dir / 'dispatch.csv', usecols=['timestamp','generation_project','DispatchGen_MW'], parse_dates=['timestamp'], infer_datetime_format=True)

#create a dictionary that matches hybrid storage to generator
hybrid_pair = pd.read_csv('generation_projects_info.csv', usecols=['GENERATION_PROJECT','storage_hybrid_generation_project'])
hybrid_pair = hybrid_pair[hybrid_pair['storage_hybrid_generation_project'] != "."]
hybrid_pair = dict(zip(hybrid_pair.GENERATION_PROJECT, hybrid_pair.storage_hybrid_generation_project))

try:
    storage = pd.read_csv(data_dir / 'storage_dispatch.csv', parse_dates=['timestamp'], infer_datetime_format=True)
    #merge charging data for hybrid storage projects into the dataframe
    hybrid_dispatch = storage.copy()[['generation_project','timestamp','ChargeMW','DispatchMW']]
    hybrid_dispatch.loc[:,'generation_project'] = hybrid_dispatch['generation_project'].replace(hybrid_pair)
    dispatch = dispatch.merge(hybrid_dispatch, how='left', on=['generation_project','timestamp']).fillna(0)

    # subtract out charging energy from generation from hybrid projects to get true generation
    dispatch.loc[:,'DispatchGen_MW'] = dispatch['DispatchGen_MW'] - dispatch['ChargeMW'] + dispatch['DispatchMW']
    #drop the hybrid columns
    dispatch = dispatch.drop(columns=['ChargeMW','DispatchMW'])
    # rename the column
    dispatch = dispatch.rename(columns={'DispatchGen_MW':'DispatchMW'})

    #append storage 
    storage_dispatch = storage[['generation_project','timestamp','DispatchMW']]
    storage_dispatch = storage_dispatch[~storage_dispatch['generation_project'].str.contains('HYBRID')]
    dispatch = dispatch.append(storage_dispatch)

    # Setup storage charging timeseries data
    ########################################
    storage_charge = storage[['generation_project','timestamp','ChargeMW']]
    storage_charge = storage_charge[~storage_charge['generation_project'].str.contains('HYBRID')]
    storage_charge = storage_charge.groupby('timestamp').sum().reset_index()

except FileNotFoundError:
    #calculate total generation
    dispatch = dispatch.rename(columns={'DispatchGen_MW':'DispatchMW'})

#add system power data if available
try:
    system_power_use = pd.read_csv(data_dir / 'system_power.csv', usecols=['timestamp','System_Power_MW'], parse_dates=['timestamp'], infer_datetime_format=True)
    system_power_use.loc[:,'generation_project'] = 'SYSTEM_POWER'
    system_power_use = system_power_use.rename(columns={'System_Power_MW':'DispatchMW'})
    dispatch = dispatch.append(system_power_use)

except FileNotFoundError:
    pass

#add technology column
dispatch['Technology'] = [i.split('_')[0] for i in dispatch['generation_project']]

# group the data by technology type
dispatch_by_tech = dispatch.groupby(['Technology','timestamp']).sum().reset_index()
#only keep observations greater than 0
dispatch_by_tech = dispatch_by_tech[dispatch_by_tech['DispatchMW'] > 0]
# Setup load timeseries data
############################

load = pd.read_csv(data_dir / 'load_balance.csv', usecols=['timestamp','zone_demand_mw'], parse_dates=['timestamp'], infer_datetime_format=True)

# Create Figure
###############

color_map = {'HYDRO':'Purple',
 'ONWIND':'Blue',
 'OFFWIND':'Navy',
 'PV':'Yellow',
 'PVHYBRID':'GreenYellow',
 'CSP':'Orange',
 'GEO':'Sienna',
 'STORAGE':'Green',
 'STORAGEHYBRID':'GreenYellow',
 'SYSTEM':'Red'}

dispatch_fig = px.area(dispatch_by_tech, x='timestamp', y='DispatchMW', color='Technology', color_discrete_map=color_map, labels={'DispatchMW':'MW','timestamp':'Datetime','Technology':'Key'})
dispatch_fig.update_traces(line={'width':0})
#dispatch_fig.update_traces(hoveron="points+fills")
dispatch_fig.layout.template = 'plotly_white'
#hoveron = 'points+fills'

try:
    # add charging line
    dispatch_fig.add_scatter(x=storage_charge.timestamp, y=(storage_charge.ChargeMW + load.zone_demand_mw), line=dict(color='green', width=4), name='Load + Storage Charge')
except NameError:
    pass

# add load line
dispatch_fig.add_scatter(x=load.timestamp, y=load.zone_demand_mw, line=dict(color='black', width=4), name='Demand')

dispatch_fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1d", step="day", stepmode="backward"),
            dict(count=7, label="1w", step="day", stepmode="backward"),
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(step="all")
        ])))

dispatch_fig.show()

In [ ]:
# Battery State of Charge data
##############################

try:

    storage = pd.read_csv(data_dir / 'storage_dispatch.csv', parse_dates=['timestamp'], infer_datetime_format=True)
    
    soc = storage.pivot(index='timestamp', columns='generation_project', values='StateOfCharge')
    #remove columns where all values are 0
    soc = soc.loc[:, (soc != 0).any(axis=0)]
    
    #load storage capacity
    storage_energy_capacity = pd.read_csv(data_dir / 'storage_builds.csv', usecols=['generation_project','OnlineEnergyCapacityMWh'], index_col='generation_project')

    #create another dictionary of storage energy capacity summed by storage type
    storage_energy_capacity = storage_energy_capacity.reset_index()
    storage_energy_capacity['generation_project'] = [i.split('_')[0] for i in storage_energy_capacity['generation_project']]
    storage_energy_capacity = storage_energy_capacity.groupby('generation_project').sum()
    grouped_storage_energy_capacity_dict = storage_energy_capacity.to_dict()['OnlineEnergyCapacityMWh']


    #sum by storage type
    soc.columns = [i.split('_')[0] for i in soc.columns]
    
    soc = soc.groupby(soc.columns, axis=1).sum()
    
    #divide by the total capacity to get state of charge
    soc = soc.div(soc.assign(**grouped_storage_energy_capacity_dict))
    
    #soc.index = pd.to_datetime(soc.index)
    
    soc.head(5)

    soc_fig = px.line(soc, x=soc.index, y=list(soc.columns), color_discrete_map={'STORAGE':'green','STORAGEHYBRID':'yellowgreen'}, labels={'timestamp':'Datetime','value':'%'}, title='Storage State of Charge')

    soc_fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1d", step="day", stepmode="backward"),
            dict(count=7, label="1w", step="day", stepmode="backward"),
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(step="all")
        ])))

    soc_fig.show()
    
except FileNotFoundError:
    pass

In [ ]:
mh_dispatch = dispatch.copy()
mh_dispatch = mh_dispatch.set_index('timestamp')

#add a technology column
mh_dispatch['Technology'] = mh_dispatch['generation_project'].str.split('_', expand=True)[0]

#sum by technology
mh_dispatch = mh_dispatch.groupby(['Technology', mh_dispatch.index], axis=0).sum().reset_index(level=0)

#groupby month and hour
mh_dispatch = mh_dispatch.groupby(['Technology', mh_dispatch.index.month, mh_dispatch.index.hour], axis=0).mean()
mh_dispatch.index = mh_dispatch.index.rename(['Technology','Month','Hour'])
mh_dispatch = mh_dispatch.reset_index()


mh_dispatch = mh_dispatch[mh_dispatch['DispatchMW'] > 0]


#load data
mh_load = pd.read_csv(data_dir / 'load_balance.csv', usecols=['timestamp','zone_demand_mw'], index_col='timestamp', parse_dates=True, infer_datetime_format=True).rename(columns={'zone_demand_mw':'DEMAND'})
mh_load = mh_load.groupby([mh_load.index.month, mh_load.index.hour], axis=0).mean()
mh_load.index = mh_load.index.rename(['Month','Hour'])
mh_load = mh_load.reset_index()

try:
    mh_charge = storage_charge.copy().set_index('timestamp')
    mh_charge = mh_charge.groupby([mh_charge.index.month, mh_charge.index.hour], axis=0).mean()
    mh_charge.index = mh_charge.index.rename(['Month','Hour'])
    mh_charge = mh_charge.reset_index()
    #merge load data
    mh_charge = mh_charge.merge(mh_load, how='left', on=['Month','Hour'])
    mh_charge['ChargeMW'] = mh_charge['ChargeMW'] + mh_charge['DEMAND']
except NameError:
    pass


# Generate the Figure
#####################

color_map = {'HYDRO':'Purple',
 'ONWIND':'Blue',
 'OFFWIND':'Navy',
 'PV':'Yellow',
 'PVHYBRID':'GreenYellow',
 'CSP':'Orange',
 'GEO':'Sienna',
 'STORAGE':'Green',
 'STORAGEHYBRID':'GreenYellow',
 'SYSTEM':'Red'}

mh_fig = px.area(mh_dispatch, x='Hour', y='DispatchMW', facet_col='Month', color='Technology', line_group='Technology', color_discrete_map=color_map, facet_col_wrap=6, width=1000, height=600, title='Month-Hour Average Generation Profiles')
mh_fig.layout.template = 'plotly_white'
mh_fig.update_traces(line={'dash':'dot'})

try:
    mh_fig.add_scatter(x=mh_charge.loc[mh_charge['Month'] == 1, 'Hour'], y=mh_charge.loc[mh_charge['Month'] == 1, 'ChargeMW'], line=dict(color='green', width=4), row=2, col=1, name='STORAGE_Charge', showlegend=True)
    mh_fig.add_scatter(x=mh_charge.loc[mh_charge['Month'] == 2, 'Hour'], y=mh_charge.loc[mh_charge['Month'] == 2, 'ChargeMW'], line=dict(color='green', width=4), row=2, col=2, name='STORAGE_Charge', showlegend=False)
    mh_fig.add_scatter(x=mh_charge.loc[mh_charge['Month'] == 3, 'Hour'], y=mh_charge.loc[mh_charge['Month'] == 3, 'ChargeMW'], line=dict(color='green', width=4), row=2, col=3, name='STORAGE_Charge', showlegend=False)
    mh_fig.add_scatter(x=mh_charge.loc[mh_charge['Month'] == 4, 'Hour'], y=mh_charge.loc[mh_charge['Month'] == 4, 'ChargeMW'], line=dict(color='green', width=4), row=2, col=4, name='STORAGE_Charge', showlegend=False)
    mh_fig.add_scatter(x=mh_charge.loc[mh_charge['Month'] == 5, 'Hour'], y=mh_charge.loc[mh_charge['Month'] == 5, 'ChargeMW'], line=dict(color='green', width=4), row=2, col=5, name='STORAGE_Charge', showlegend=False)
    mh_fig.add_scatter(x=mh_charge.loc[mh_charge['Month'] == 6, 'Hour'], y=mh_charge.loc[mh_charge['Month'] == 6, 'ChargeMW'], line=dict(color='green', width=4), row=2, col=6, name='STORAGE_Charge', showlegend=False)
    mh_fig.add_scatter(x=mh_charge.loc[mh_charge['Month'] == 7, 'Hour'], y=mh_charge.loc[mh_charge['Month'] == 7, 'ChargeMW'], line=dict(color='green', width=4), row=1, col=1, name='STORAGE_Charge', showlegend=False)
    mh_fig.add_scatter(x=mh_charge.loc[mh_charge['Month'] == 8, 'Hour'], y=mh_charge.loc[mh_charge['Month'] == 8, 'ChargeMW'], line=dict(color='green', width=4), row=1, col=2, name='STORAGE_Charge', showlegend=False)
    mh_fig.add_scatter(x=mh_charge.loc[mh_charge['Month'] == 9, 'Hour'], y=mh_charge.loc[mh_charge['Month'] == 9, 'ChargeMW'], line=dict(color='green', width=4), row=1, col=3, name='STORAGE_Charge', showlegend=False)
    mh_fig.add_scatter(x=mh_charge.loc[mh_charge['Month'] == 10, 'Hour'], y=mh_charge.loc[mh_charge['Month'] == 10, 'ChargeMW'], line=dict(color='green', width=4), row=1, col=4, name='STORAGE_Charge', showlegend=False)
    mh_fig.add_scatter(x=mh_charge.loc[mh_charge['Month'] == 11, 'Hour'], y=mh_charge.loc[mh_charge['Month'] == 11, 'ChargeMW'], line=dict(color='green', width=4), row=1, col=5, name='STORAGE_Charge', showlegend=False)
    mh_fig.add_scatter(x=mh_charge.loc[mh_charge['Month'] == 12, 'Hour'], y=mh_charge.loc[mh_charge['Month'] == 12, 'ChargeMW'], line=dict(color='green', width=4), row=1, col=6, name='STORAGE_Charge', showlegend=False)
except (NameError, KeyError) as e:
    pass

mh_fig.add_scatter(x=mh_load.loc[mh_load['Month'] == 1, 'Hour'], y=mh_load.loc[mh_load['Month'] == 1, 'DEMAND'], line=dict(color='black', width=4), row=2, col=1, name='Demand', showlegend=True)
mh_fig.add_scatter(x=mh_load.loc[mh_load['Month'] == 2, 'Hour'], y=mh_load.loc[mh_load['Month'] == 2, 'DEMAND'], line=dict(color='black', width=4), row=2, col=2, name='Demand',showlegend=False)
mh_fig.add_scatter(x=mh_load.loc[mh_load['Month'] == 3, 'Hour'], y=mh_load.loc[mh_load['Month'] == 3, 'DEMAND'], line=dict(color='black', width=4), row=2, col=3, name='Demand', showlegend=False)
mh_fig.add_scatter(x=mh_load.loc[mh_load['Month'] == 4, 'Hour'], y=mh_load.loc[mh_load['Month'] == 4, 'DEMAND'], line=dict(color='black', width=4), row=2, col=4, name='Demand', showlegend=False)
mh_fig.add_scatter(x=mh_load.loc[mh_load['Month'] == 5, 'Hour'], y=mh_load.loc[mh_load['Month'] == 5, 'DEMAND'], line=dict(color='black', width=4), row=2, col=5, name='Demand', showlegend=False)
mh_fig.add_scatter(x=mh_load.loc[mh_load['Month'] == 6, 'Hour'], y=mh_load.loc[mh_load['Month'] == 6, 'DEMAND'], line=dict(color='black', width=4), row=2, col=6, name='Demand', showlegend=False)
mh_fig.add_scatter(x=mh_load.loc[mh_load['Month'] == 7, 'Hour'], y=mh_load.loc[mh_load['Month'] == 7, 'DEMAND'], line=dict(color='black', width=4), row=1, col=1, name='Demand', showlegend=False)
mh_fig.add_scatter(x=mh_load.loc[mh_load['Month'] == 8, 'Hour'], y=mh_load.loc[mh_load['Month'] == 8, 'DEMAND'], line=dict(color='black', width=4), row=1, col=2, name='Demand', showlegend=False)
mh_fig.add_scatter(x=mh_load.loc[mh_load['Month'] == 9, 'Hour'], y=mh_load.loc[mh_load['Month'] == 9, 'DEMAND'], line=dict(color='black', width=4), row=1, col=3, name='Demand', showlegend=False)
mh_fig.add_scatter(x=mh_load.loc[mh_load['Month'] == 10, 'Hour'], y=mh_load.loc[mh_load['Month'] == 10, 'DEMAND'], line=dict(color='black', width=4), row=1, col=4, name='Demand', showlegend=False)
mh_fig.add_scatter(x=mh_load.loc[mh_load['Month'] == 11, 'Hour'], y=mh_load.loc[mh_load['Month'] == 11, 'DEMAND'], line=dict(color='black', width=4), row=1, col=5, name='Demand', showlegend=False)
mh_fig.add_scatter(x=mh_load.loc[mh_load['Month'] == 12, 'Hour'], y=mh_load.loc[mh_load['Month'] == 12, 'DEMAND'], line=dict(color='black', width=4), row=1, col=6, name='Demand', showlegend=False)

month_names = ['July', 'August', 'September', 'October', 'November', 'December','January', 'February', 'March', 'April', 'May', 'June']
for i, a in enumerate(mh_fig.layout.annotations):
    a.text = month_names[i]


mh_fig.update_xaxes(dtick=3)
mh_fig.show()

In [ ]:
# get month-hour shape of open energy position
##############################################

dispatch_gen = pd.read_csv(data_dir / 'dispatch.csv', usecols=['timestamp','DispatchGen_MW'], parse_dates=['timestamp'], infer_datetime_format=True)

#sum by timestamp
dispatch_gen = dispatch_gen.groupby('timestamp').sum().reset_index()

#sum to get total generation
dispatch_gen['Generation_MW'] = dispatch_gen['DispatchGen_MW']

#merge generation_load_mismatch data
generation_load_mismatch = pd.read_csv(data_dir / 'load_balance.csv', usecols=['timestamp','zone_demand_mw'], parse_dates=['timestamp'], infer_datetime_format=True)

generation_load_mismatch = generation_load_mismatch.merge(dispatch_gen[['timestamp','Generation_MW']], how='left', on='timestamp')

#subtract generation from generation_load_mismatch to get open position and replace negative values with zero
generation_load_mismatch['Overgeneration_MW'] = generation_load_mismatch['Generation_MW'] - generation_load_mismatch['zone_demand_mw']
#generation_load_mismatch.loc[generation_load_mismatch['generation_load_mismatch_MW'] < 0, 'generation_load_mismatch_MW'] = 0

#generation_load_mismatch['timestamp'] = pd.to_datetime(generation_load_mismatch['timestamp'])

generation_load_mismatch = generation_load_mismatch.set_index('timestamp')

generation_load_mismatch_mh = generation_load_mismatch.groupby([generation_load_mismatch.index.month, generation_load_mismatch.index.hour], axis=0).mean()
generation_load_mismatch_mh.index = generation_load_mismatch_mh.index.rename(['Month','Hour'])
generation_load_mismatch_mh = generation_load_mismatch_mh.reset_index()

mh_mismatch_fig = px.area(generation_load_mismatch_mh, x='Hour', y='Overgeneration_MW', facet_col='Month', facet_col_wrap=6, width=1000, height=600, title='Shape of over- and under-generation, excluding energy storage')
mh_mismatch_fig.update_xaxes(dtick=3)
month_names = ['July', 'August', 'September', 'October', 'November', 'December','January', 'February', 'March', 'April', 'May', 'June']
for i, a in enumerate(mh_mismatch_fig.layout.annotations):
    a.text = month_names[i]

mh_mismatch_fig.show()



## Battery Cycles

In [ ]:
try:
    cycles = pd.read_csv(data_dir / 'storage_cycle_count.csv', usecols=['generation_project','storage_max_annual_cycles','Battery_Cycle_Count'])
    cycles = cycles.round(decimals=2)
    cycles = cycles[cycles['Battery_Cycle_Count'] > 0]
    storage_energy_capacity = pd.read_csv(data_dir / 'storage_builds.csv', usecols=['generation_project','OnlineEnergyCapacityMWh'])
    cycles = cycles.merge(storage_energy_capacity, how='left', on='generation_project')
    cycles['Battery_Cycle_Count'] = cycles['Battery_Cycle_Count'] / cycles['OnlineEnergyCapacityMWh']
    cycles = cycles.drop(columns=['OnlineEnergyCapacityMWh']).set_index('generation_project').round(decimals=0)
    cycles['Battery_Cycle_Count'] = cycles['Battery_Cycle_Count'].astype(int)
except FileNotFoundError:
    cycles = "No batteries in model"
cycles